In [ ]:
# %pip install hyperopt

In [9]:
#import packages and soil moisture water balance function
%reload_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import glob
import geopandas as gpd
import os
import pandas as pd
import sys
sys.path.insert(0, '../../PixSWAB_V1')
import calibration_tool.pixswab_v1_calibration_for_hyperOpt as pixswab
import calibration_tool.hydroStats as hs

import datetime
import time
# import gc
from csv import DictWriter
from hyperopt.pyll.base import scope
from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
from hyperopt import hp
import ast

# Pixswab Optimization

In [10]:
# start a cleint for distributed computing for files larger than the memory capacity (RAM)
# from dask.distributed import Client, LocalCluster
# import multiprocessing as mp
# from dask.distributed import progress

# # multiprocessing client
# client = pixswab.start_multiprocessing()
# client.restart()

In [18]:
# get the config file and prepare inputs
config_path = r'..\config_files\pixswab_config.ini'
pixswab_inputs, pixswab_parameters, log_file_path, out_dir, encoding_output, bsahpe = pixswab.prepare_inputs(config_path)

# read observed ourflw (discharge)
Qobs_file = r'..\..\Upper_Litani\input\observed_flow\discharge_Upper_Litani_JoubJannine_beforeQaraoun.csv'
Qobs=pd.read_csv(Qobs_file,sep=',',index_col=0)
target_column = 'km3/month'
Qobs = Qobs[target_column]
Qobs.index = pd.to_datetime(Qobs.index)

# Path of calibration result file
cal_path = r'..\..\Upper_Litani\calibration_results\calibration.csv'
def write_to_csv_results(new_row,cal_path):
    with open(cal_path, 'a', newline='') as f_object:
        # Pass the file object and a list of column names to DictWriter()
        dictwriter_object = DictWriter(f_object, fieldnames=list(new_row.keys()))
        # Pass the dictionary as an argument to the Writerow()
        dictwriter_object.writerow(new_row)
        # Close the file object
        f_object.close()

# HyperOpt

In [19]:
WarmupDays = 0
space = {
    'baseFlowReces': hp.uniform('baseFlowReces',0.0,1.0),
    'deepPerc': hp.uniform('deepPerc',0.0,1.0),
    'rcExp':hp.uniform('rcExp',1, 5.),
    }

def f_nn(params):
    
    result = pixswab.run_pixswab(pixswab_inputs, params)
    Qsim = pixswab.compute_discharge(result, bsahpe, out_dir, log_file_path)
    Qsim_target_column = 'discharge'
    Qsim = Qsim[Qsim_target_column]
    Qsim = Qsim[Qsim.index.isin(Qobs.index)]
    KGE = hs.KGE(s=Qsim, o=Qobs, warmup=WarmupDays)
    NSE = hs.NS(s=Qsim, o=Qobs, warmup=WarmupDays)
    rmse = hs.rmse(s=Qsim, o=Qobs, warmup=WarmupDays)
    # print('KGE = {0}, NSE = {1}, rmse = {2}'.format(KGE, NSE, rmse))
      
    new_row = {
            'algorithm_name':'hyperopt',
            'KGE': KGE,
            'NSE': NSE,
            'rmse': rmse,
            'baseFlowReces': params['baseFlowReces'], 
            'deepPerc': params['deepPerc'],
            'rcExp' : params['rcExp']
            }
    write_to_csv_results(new_row,cal_path)
    return {
            # 'loss': error_results['kge']*-1,
            'loss': KGE*-1,
            'status': STATUS_OK,
            'params': params
    }

In [ ]:
import warnings
warnings.filterwarnings('ignore')
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)

 31%|██████████████▌                                | 31/100 [21:37<48:03, 41.80s/trial, best loss: -0.782664470596379]

In [ ]:
best_error = trials.results[np.argmin([r['loss'] for r in trials.results])]['loss']
best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']

worst_error = trials.results[np.argmax([r['loss'] for r in trials.results])]['loss']
worst_params = trials.results[np.argmax([r['loss'] for r in trials.results])]['params']

In [53]:
best_error

-0.8139443541372209

In [ ]:
best_params

In [27]:
error_results = pd.read_csv(os.path.join(MAIN_DIR,'calibration/calibration.csv'))

In [ ]:
error_results.sort_values(by='kge',ascending=False)

In [ ]:
error_results[40:].sort_values(by='kge',ascending=False)

In [ ]:
error_results.sort_values(by='kge',ascending=False)